In [1]:
import numpy as np
from gmr import GMM, MVN

# Create simple 2D data
np.random.seed(42)
x = np.column_stack([
    np.concatenate([np.random.uniform(-1, -0.5, 50), np.random.uniform(0.5, 1, 50)]),
    np.concatenate([np.random.uniform(0.5, 1, 50), np.random.uniform(0.5, 1, 50)])
])

# Fit GMM
gmm = GMM(n_components=2, random_state=42)
gmm.from_samples(x)

# This fails
y_target = 0.75
conditional = gmm.condition([1], [[y_target]])  # ValueError

ValueError: setting an array element with a sequence.

In [2]:
from gmr import MVN

def sample_gmm_conditional(gmm, cond_indices, cond_values, n_samples=1000):
      """Workaround: condition each component separately and mix."""
      cond_values = np.atleast_1d(cond_values)

      conditional_means = []
      conditional_covs = []
      weights = []

      for k in range(gmm.n_components):
          mvn = MVN(mean=gmm.means[k], covariance=gmm.covariances[k])
          cond_mvn = mvn.condition(cond_indices, cond_values)
          conditional_means.append(cond_mvn.mean)
          conditional_covs.append(cond_mvn.covariance)

          marginal = mvn.marginalize(cond_indices)
          likelihood = marginal.to_probability_density(cond_values.reshape(1, -1))[0]
          weights.append(gmm.priors[k] * likelihood)

      weights = np.array(weights)
      weights = weights / weights.sum()

      samples = []
      component_counts = np.random.multinomial(n_samples, weights)
      for k in range(gmm.n_components):
          if component_counts[k] > 0:
              mean = conditional_means[k]
              cov = conditional_covs[k]
              if np.isscalar(cov):
                  cov = np.array([[cov]])
              samples.append(np.random.multivariate_normal(mean.flatten(), cov, component_counts[k]))

      return np.vstack(samples)

# This works
x_samples = sample_gmm_conditional(gmm, [1], y_target, n_samples=1000)